In [2]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

ModuleNotFoundError: No module named 'numpy_financial'

## Group Assignment
### Team Number: XX
### Team Member Names:
### Team Strategy Chosen: ____Risk-Free______(Market Beat, Market Meet, Risk-Free)

In [ ]:
# Our Goal is to build a portfolio with minimal movement during Nov 21-28, 2025

# STEP 1: DATA COLLECTION & FILTERING
# approach:
# fetch historical price data (Oct 2024 - Sep 2025) using yfinance
# filter the stocks with average daily volume >= XXX shares
# calculate daily returns for each stock
# calculate beta relative to market index 
# identify low-beta stocks (beta close to 0 or slightly negative preferred)
# calculate standard deviation of returns (volatility measure)
# calculate correlation with market indices
# ... etc

import pandas as pd
import numpy as np
import yfinance as yf

tickers_df = pd.read_csv('Tickers_Example.csv')

print(len(tickers_df))
print("")
print(tickers_df.head())

start_date = '2024-10-01'
end_date = '2025-09-30'

ticker_list = tickers_df.iloc[:, 0].tolist() # first column contains tickers
data = yf.download(ticker_list, start=start_date, end=end_date, progress=False)

# extract volume data
volume_data = data['Volume'].copy()

# drop months with < 18 trading days (per assignment requirement)
volume_data.index = pd.to_datetime(volume_data.index)
monthly_counts = volume_data.resample('M').count()

for ticker in volume_data.columns:
    months_to_drop = monthly_counts[ticker][monthly_counts[ticker] < 18].index
    for month in months_to_drop:
        mask = (volume_data.index.year == month.year) & (volume_data.index.month == month.month)
        volume_data.loc[mask, ticker] = np.nan

# calculate average daily volume (excluding dropped months)
avg_volumes = volume_data.mean()

# only need stocks with volume >= 5000 
volume = 5000
filtered_tickers = avg_volumes[avg_volumes >= volume].index.tolist()

# how many we kept 
print(f"{len(filtered_tickers)} out of {len(ticker_list)} tickers")

filtered_volumes = avg_volumes[filtered_tickers].sort_values(ascending=False)
print("\nTop 5 stocks by volume:")
print(filtered_volumes.head())

# Returns, Beta, Volatility and Correlation Analysis

# download price data (Adj Close) for filtered tickers
price = yf.download(filtered_tickers, start=start_date, end=end_date, progress=False)

# take only the adjusted close
close = price['Close'].copy()

# remove any tickers with no price data at all
close = close.dropna(axis=1, how='all')

print("Number of tickers with valid price data:", len(close.columns))
print("Tickers:", list(close.columns))

# Calculate for daily returns
daily_returns = close.pct_change().dropna()

print("\nDaily returns (first 5 rows):")
print(daily_returns.head())

# Calculating the Beta relative to SPY (market benchmark)

# downloading the SPY data for the same period
spy_data = yf.download("SPY", start=start_date, end=end_date, progress=False)
spy_close = spy_data['Close'].copy()

spy_returns = spy_close.pct_change().dropna()

# Merge stock returns with SPY returns
combined_returns = pd.merge(daily_returns, spy_returns, left_index=True, right_index=True, how='inner')
print("These are the combined returns of stocks and SPY")
print(combined_returns.head())

# Beta values relative to the SPY
# Beta = Cov(stock, SPY) / Var(SPY)
MarketVar=combined_returns["SPY"].var()

# calculate the covariance matrix between each stock and SPY
Beta=combined_returns.cov()/MarketVar

print(Beta)
print('')
print('Cell [0,0] is meaningless (to get beta, we divided by the variance of the market!)')
print('')

# Identify the first stock
first_stock = Beta.columns[0]  
print('The Beta of', first_stock, 'is:', Beta.iat[0, -1])
print('The Beta of the Market is:', Beta.iat[-1, -1])
print('')

# Calculating Volatility
print(daily_returns.std())
print("Standard Deviations")
print(' ')

# Calculating Correlation
print(daily_returns.corr())
print("Correlation: ")
print(' ')

# Identifying low beta stocks
stocks_with_low_beta = Beta.iloc[:-1, -1]
print("Beta of all the stocks relative to the SPY")
print(stocks_with_low_beta)
print(' ')

# sorting by lowest beta
sort_low_beta = stocks_with_low_beta.sort_values(ascending=True)

print("Stocks sorted by lowest Beta:")
print(sort_low_beta)
print('')

In [ ]:
# Returns, Beta, Volatility and Correlation Analysis

# download price data (Adj Close) for filtered tickers
price = yf.download(filtered_tickers, start=start_date, end=end_date, progress=False)

# take only the adjusted close
close = price['Close'].copy()

# remove any tickers with no price data at all
close = close.dropna(axis=1, how='all')

print("Number of tickers with valid price data:", len(close.columns))
print("Tickers:", list(close.columns))

# Calculate for daily returns
daily_returns = close.pct_change().dropna()

print("\nDaily returns (first 5 rows):")
print(daily_returns.head())

# Calculating the Beta relative to SPY (market benchmark)

# downloading the SPY data for the same period
spy_data = yf.download("SPY", start=start_date, end=end_date, progress=False)
spy_close = spy_data['Close'].copy()

spy_returns = spy_close.pct_change().dropna()

# Merge stock returns with SPY returns
combined_returns = pd.merge(daily_returns, spy_returns, left_index=True, right_index=True, how='inner')
print("These are the combined returns of stocks and SPY")
print(combined_returns.head())

# Beta values relative to the SPY
# Beta = Cov(stock, SPY) / Var(SPY)
MarketVar=combined_returns["SPY"].var()

# calculate the covariance matrix between each stock and SPY
Beta=combined_returns.cov()/MarketVar

print(Beta)
print('')
print('Cell [0,0] is meaningless (to get beta, we divided by the variance of the market!)')
print('')

# Identify the first stock
first_stock = Beta.columns[0]  
print('The Beta of', first_stock, 'is:', Beta.iat[0, -1])
print('The Beta of the Market is:', Beta.iat[-1, -1])
print('')

# Calculating Volatility
print(daily_returns.std())
print("Standard Deviations")
print(' ')

# Calculating Correlation
print(daily_returns.corr())
print("Correlation: ")
print(' ')

# Identifying low beta stocks
stocks_with_low_beta = Beta.iloc[:-1, -1]
print("Beta of all the stocks relative to the SPY")
print(stocks_with_low_beta)
print(' ')

# sorting by lowest beta
sort_low_beta = stocks_with_low_beta.sort_values(ascending=True)

print("Stocks sorted by lowest Beta:")
print(sort_low_beta)
print('')


In [ ]:
# Step 2: Calculate Risk Score Based on Volatility & Beta

# First, we need to normalize the volatility and beta to a common scale (0-1)
volatility_min = volatility.min()
volatility_max = volatility.max()
volatility_normalized = (volatility - volatility_min) / (volatility - volatility.max())

# Normalize beta (we have already calculated beta)
beta_min = Beta.iloc[:-1, -1].min()
beta_max = Beta.iloc[:-1, -1].max()
beta_normalized = (Beta.iloc[:-1, -1] - beta_min) / (beta_max - beta_min)

# Calculate Risk Score: 60% volatility and 40% beta
risk_score = 0.6 * volatility_normalized + 0.4 * beta_normalized

# Create a DataFrame to store the risk score and other metrics
risk_metrics = pd.DataFrame({
    'Volatility': volatility,
    'Beta': Beta.iloc[:-1, -1],
    'RiskScore': risk_score
})

# Sort stocks by their Risk Score (ascending: lowest risk first)
sorted_risk_metrics = risk_metrics.sort_values(by='RiskScore', ascending=True)

# Display the sorted stocks (first few)
print("Sorted stocks by Risk Score (lowest risk first):")
print(sorted_risk_metrics.head())

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.